In [1]:
#%%
#this enviornment is LLM
import torch
import gc
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import mlflow

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 加载 GPT-2 模型和分词器
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


# 設置 tokenizer 為左側填充
tokenizer.padding_side = 'left'

'''
texts = "how are you?"

input = tokenizer(texts, return_tensors="pt", padding=False)

print(input)

input = input
#output = tokenizer.decode(input['input_ids'], skip_special_tokens=True)
output = tokenizer.decode(4919, skip_special_tokens=True)
'''


# 输入文本
texts = ["The first time I saw the film was in the early '90s. Let's continue the story in English.","Gigi Hadid is beautiful. "]

# 将文本转换为模型输入
tokenizer.pad_token = tokenizer.eos_token
input = tokenizer(texts, return_tensors="pt", padding=True, return_attention_mask=True)#, if texts is a list
#inputs_ids = will be a list of a token_id(for each word in one sentences) 
#inputs 里会包含 attention_mask
input.to(device)
input_ids = input['input_ids']
attention_mask = input['attention_mask']

#print(input_ids)
#print(attention_mask)
# 使用模型生成文本
model.to(device)
##clean_up_tokenization_spaces清除額外空格
##max_new_tokens：新生成的 token 的最大數量
output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.7,  # 控制生成的隨機性
    max_new_tokens=50,  # 控制生成長度
    top_p=0.9,  # 使用 nucleus sampling
    top_k=50,  # 考慮前 50 個 token
    repetition_penalty=2.0,
    pad_token_id=tokenizer.eos_token_id,  # 填充 token
    eos_token_id=tokenizer.eos_token_id,  # 結束 token
    num_return_sequences=1
    #clean_up_tokenization_spaces=True   #if use generator in pipeline or mayber gpt-4 than set the value
)
##num_return_sequences=1
#print(output.shape)
#print(len(output))
for i in range(len(output)):##沒有一次generate的嗎?
    '''
    print(output[i].shape)
    generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
    '''
    generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
    # on purpose of to make sure that a sentences end in some certain  Marks.
    if not generated_text[-1] in ".!?":
        last_punctuation = max(generated_text.rfind(p) for p in ".!?")
        if last_punctuation != -1:
            generated_text = generated_text[:last_punctuation + 1]
    print(f"{i} sentences: {generated_text}")




/home/kaia/miniforge3/envs/LLM/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


0 sentences: The first time I saw the film was in the early '90s. Let's continue the story in English. The idea is to see a man who has been arrested for being gay, and he comes out as bisexual.
1 sentences: Gigi Hadid is beautiful.  She's a goddess, and she loves to play with her dolls!


In [2]:
#finetune:


#the model which pretrain to save in the local
model.save_pretrained('./gpt2_local_model')
tokenizer.save_pretrained('./gpt2_local_model')

from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('./gpt2_local_model')
model = GPT2LMHeadModel.from_pretrained('./gpt2_local_model')

In [3]:
torch.cuda.empty_cache()
gc.collect()

118